In [889]:
class GGH:

    def __init__(self, d, privateKey = None, privateMatrix = None):
        self.__d = d
        self.__privateKey = privateKey
        self.__privateMatrix = privateMatrix
        if privateKey != None:
            self.setKey(privateKey, privateMatrix)
        else:
            self.generateKey(d)

    def setKey(self, privateKey, privateMatrix = None):
        if 0.9 <= hadamardRatio(privateKey) <= 1:
                self.__privateKey = privateKey

        if privateMatrix != None:
            if privateKey.nrows() == privateMatrix.nrows() and privateKey.ncols() == privateMatrix.ncols():
                self.__privateMatrix = privateMatrix
            else:
                raise Exception("Private keys must be the same size")

        self.generatePublicKey()

    def generateKey(self, d):
        self.generatePrivateKey(d)
        self.generatePublicKey()

    def generatePrivateKey(self, d):
        while True:
            key = random_matrix(ZZ, d, x=-128, y=128)
            try:
                if hadamardRatio(key) >= 0.9:
                    self.__privateKey = key
                    break
            except ZeroDivisionError:
                pass

    def generatePublicKey(self):
        if self.__privateMatrix == None:
            self.__privateMatrix = random_matrix(ZZ, self.__d, upper_bound=128, algorithm='unimodular')
            key = self.__privateMatrix * self.__privateKey
            while hadamardRatio(key) > 0.1:
                self.__privateMatrix = random_matrix(ZZ, self.__d, upper_bound=128, algorithm='unimodular')
                key = self.__privateMatrix * self.__privateKey
        
            self.__publicKey = key
        else:
            self.__publicKey = self.__privateMatrix * self.__privateKey

    def encrypt(self, m, e = None):
        message_length = len(m)
        if e == None:
            sigma = floor(shortestVector(self.__privateKey)/2)
            if sigma >= 5 or sigma <= 0:
                sigma = 5
            e = random_vector(ZZ, message_length, x=-sigma, y=sigma)
        return (m * self.__publicKey) + e

    def decrypt(self, c): 
        mW = babai(c, self.__privateKey)
        return mW * self.__publicKey.inverse()

    def encryptString(self, m):
        if type(m) == str:
            n = self.__privateKey.nrows()
            messages = [m[i:i+n] for i in range(0, len(m), n)]
            cipher = []
            for block in messages:
                while len(block) < n: 
                    block += chr(0)
                cipher.append(self.encrypt(vector([ord(c) for c in block])))
            return cipher
        else:
            raise Exception("Message must be a string")

    def decryptString(self, c):
        message = ""
        for i in c:
            block = self.decrypt(vector(i))
            for m in block:
                message += chr(m)
        return message
    
    def getPublicKey(self):
        return self.__publicKey

def babai(w, basis):
    r = Matrix(RR, w)
    R = basis.change_ring(RR).solve_left(r)
    z = vector(ZZ, [round(t) for t in R[0]])
    return z * basis
    
def hadamardRatio(basis):
    if basis.nrows() != basis.ncols():
        raise Exception("Basis must be square")
    basis = basis.change_ring(RR)
    d = basis.nrows()
    topValue = basis.determinant().abs()
    bottomValue = 1
    for i in basis:
        bottomValue *= i.norm()
    return (topValue/bottomValue) ** (1/d)

def shortestVector(basis):
    basis.change_ring(RR)
    min = float('inf')
    for x in basis:
        if x.norm() < min:
            min = x.norm()
    return min

The first example is given the private key and randomly generating a public key.

In [890]:
privateKey = Matrix(ZZ, 2, 2, [[7, 0], [0, 3]])
ggh = GGH(2, privateKey)
print("The public key is:")
print(ggh.getPublicKey())

The public key is:
[-77 -57]
[-28 -21]


After generating the public key, the sender can use the public key to encrypt a message.

In [891]:
m = vector(ZZ, [3, -7])
print(f"The plaintext message is: {m}")
c = ggh.encrypt(m)
print(f"The ciphertext message is: {c}")

The plaintext message is: (3, -7)
The ciphertext message is: (-35, -25)


Once the receiver received the ciphertext, he can use his private key to decrypt the message.

In [892]:
ggh.decrypt(c)
print(f"The decrypted plaintext message is: {m}")

The decrypted plaintext message is: (3, -7)


The next example is randomly generating a private key in a given dimensional size (4 in this case).

In [893]:
ggh2 = GGH(4)
print("The public key is:")
ggh2.getPublicKey()

The public key is:


[ 9634 -8919  1363  6234]
[10117 -8954  1409  6700]
[ 3166 -2808   354  2336]
[-4102  3763  -575 -2676]

We can encrypt a message by translating the characters into their ASCII values, and then do the regular decryption.

In [894]:
m = "Hello, World!"
print(f"The plaintext message is \"{m}\"")
cc = ggh2.encryptString(m)
print("The ciphertext message is:")
print(cc)

The plaintext message is "Hello, World!"
The ciphertext message is:
[(1614377, -1443381, 216576, 1088830), (1258959, -1146464, 174589, 828717), (2154436, -1937733, 292646, 1440458), (317924, -294323, 44982, 205724)]


We can decrypt each block of ciphertext and translate them from ASCII to English.

In [895]:
m = ggh2.decryptString(cc)
print(f"The decrypted plaintext message is \"{m}\"")

The decrypted plaintext message is "Hello, World!   "


The blow class defines three possible methods to attack the GGH cryptosystem.

In [896]:
class GGHAttacker:
    
    def __init__(self, publicKey):
        self.__publicKey = publicKey

    def solvePrivateKey(self):
        return self.__publicKey.LLL()

    def solveErrorVector(self, c):
        d = len(c) + 1
        entries = [[j for j in i] + [0] for i in self.__publicKey]
        entries += ([[i for i in c] + [1]])
        newMatrix = matrix(ZZ, d, d, entries)
        row = [i for i in newMatrix.LLL()[0]]
        error = row[0: d - 1]
        errorVector = vector(ZZ, error)
        print(f"The error vector is: {errorVector}\n")
        return (c - errorVector) * self.__publicKey.inverse()

    def nguyenAttack(self, c, sigma):
        left = (c + vector(ZZ, [sigma] * len(c))) % (2 * sigma)
        right = self.__publicKey % (2 * sigma)
        return right.solve_left(left)
        


The first method is we can use the LLL algorithm to reduce the public key into a short, orthogonal basis (a good basis like the private key).

In [897]:
attacker = GGHAttacker(ggh.getPublicKey())
print("The private key in this system is")
print(privateKey)
print("The LLL reduced matrix is")
print(attacker.solvePrivateKey())

The private key in this system is
[7 0]
[0 3]
The LLL reduced matrix is
[0 3]
[7 0]


In [898]:
m = vector(ZZ, [3, -7])
c = ggh.encrypt(m)
m = babai(c, attacker.solvePrivateKey())
m * ggh.getPublicKey().inverse()

(3, -7)

Also, we can test that as long as the basis is nearly orthogonal, we can use LLL algorithm to reduce the public key to a similar private key.

In [899]:
privateKey = None
while True:
    key = random_matrix(ZZ, 2, x=-128, y=128)
    try:
        if hadamardRatio(key) >= 0.9:
            privateKey = key
            break
    except ZeroDivisionError:
        pass

privateMatrix = random_matrix(ZZ, 2, upper_bound=128, algorithm='unimodular')

m = vector(ZZ, [5, 2])
ggh_5 = GGH(2, privateKey)
print("The private key in this system is")
print(privateKey)
c = ggh_5.encrypt(m)
attacker_5 = GGHAttacker(ggh_5.getPublicKey())
print("The LLL reduced matrix is")
print(attacker_5.solvePrivateKey())
babai(c, attacker_5.solvePrivateKey()) * ggh_5.getPublicKey().inverse()

The private key in this system is
[ 75  35]
[ 12 -16]
The LLL reduced matrix is
[-12  16]
[-63 -51]


(5, 2)

The second attack method is to construct a new matrix with known information such as ciphertext and the public key. Then, we can compute the error vector used in the encryption process.

In [900]:
b = matrix(ZZ, 3, 3, [[-97, 19, 19], [-36, 30, 86], [-184, -64, 78]])
u = matrix(ZZ, 3, 3, [[4327, -15447, 23454], [3297, -11770, 17871], [5464, -19506, 29617]])
m = vector(ZZ, [86, -35, -32])
e = vector(ZZ, [-4, -3, 2])
ggh_2 = GGH(3, b, u)
c = ggh_2.encrypt(m, e)

In [901]:
attacker_2 = GGHAttacker(ggh_2.getPublicKey())
print(f"The decrypted plaintext message is: {attacker_2.solveErrorVector(c)}")

The error vector is: (-4, -3, 2)

The decrypted plaintext message is: (86, -35, -32)


The third attack method is Nguyen's Attack which uses a flaw in its design.

In [902]:
ggh_3 = GGH(2, matrix(ZZ, 2, [[17, 0], [0, 19]]), matrix(ZZ, 2, [[2, 3], [3, 5]]))
m = vector(ZZ, [2, -5])
e = vector(ZZ, [1, -1])
c = ggh_3.encrypt(m, e)

In [903]:
attacker_3 = GGHAttacker(ggh_3.getPublicKey())
sigma = 1
print(f"The decrypted plaintext message is: {attacker_3.nguyenAttack(c, sigma)} mod {2 * sigma}")

The decrypted plaintext message is: (0, 1) mod 2
